# 🎓 Personalized Career Coach

---

## 📌 Problem Statement
Choosing the right career path is one of the most important and difficult decisions students face.
Many lack access to personalized guidance or clarity about which skills align with specific industries and 
opportunities.

---

## 🎯 Solution
This project introduces a **Personalized Career Coach**, powered by **OpenAI GPT-3.5**, that helps students:
- Get **tailored career suggestions** based on interests, skills, and goals.
- Receive **hand-picked course recommendations** from any trusted platforms.
- **Optionally upload a resume** to generate insights from real-world experience.
- View suggestions as **clickable cards** to explore further.
- Export a personalized **PDF career report**

---

## 🧠 How it Works
1. The student enters:
   - Name and age
   - Interests, skills, and goals
   - Or uploads a resume (PDF or TXT)

2. The app:
   - Uses GPT-3.5 to recommend 2-3 career paths with justifications
   - Then suggests 1 online course for each path
   - Renders the course suggestions as **interactive HTML cards**
   - Offers a clean, downloadable **PDF Summary** with clickable links

---

## 🛠️ Technology Used
- **OpenAI GPT-3.5 Turbo** for reasoning and recommendations.
- **FAISS + TF-IDF** for simple RAG-style grounding.
- **Gradio** for the user-friendly web interface.
- **PyMuPDF** for extracting text from resumes.
- **FPDF** to export structured PDF report.
- **HTML rendering** for clickable, styled course cards.

---

## ✨ Generative AI Capabilities Demonstrated
✅ **Few-short prompting**

✅ **Multi-step agent reasoning**

✅ **Resume text understanding**

✅ **Structured HTML output**

✅ **Vector Search with FAISS (for RAG grounding)**

---

## ✅ Try it Yourself
Scroll down and:
- Fill out the profile or upload your resume.
- Hit **Submit**.
- Click any card to explore your personalized career roadmap and a downloadable PDF to take with you! 🎓🚀

In [1]:
# 1. Install dependencies
!pip install openai -q
!pip install faiss-cpu -q
!pip install gradio -q
!pip install pymupdf -q
!pip install fpdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 22.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 35.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 88.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 66.7 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done


### 📚 2: Import Libraries
We import standard libraries for:
- Resume Writing.
- Vector Search.
- UI development.
- Working with the OpenAI API

In [2]:
import os
import re
import json
import fitz
import faiss
import numpy as np
import gradio as gr
from kaggle_secrets import UserSecretsClient
from sklearn.feature_extraction.text import TfidfVectorizer
from openai import OpenAI
from fpdf import FPDF

### 🔐 3: Authenticate with OpenAI
We securely load the OpenAI API key from Kaggle Secrets and initialize the `OpenAI` client.

This allows us to generate career and course suggestions using GPT-3.5

In [3]:
user_secrets = UserSecretsClient()
client = OpenAI(api_key=user_secrets.get_secret("OPENAI_API_KEY"))

### 🔍 4. Build Vector Search Index
I used `FAISS` and `TF-IDF` to create a searchable index of career profiles.

This step isn't essential for core functionality, but demonstrates how to ground Gen AI with domain-specific documents.

In [4]:
career_docs = [
    "Clinical Psychologist: Help patients manage emotional and mental disorders.",
    "Science Writer: Communicate scientific research through engaging articles.",
    "Healthcare Educator: Teach communities about health and wellness.",
    "Biomedical Researcher: Investigate human biology and diseases.",
    "Speech Therapist: Support individuals with communication difficulties."
]
vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(career_docs).toarray()
index = faiss.IndexFlatL2(doc_vectors.shape[1])
index.add(np.array(doc_vectors).astype("float32"))

### 📝 5: Resume Upload & Text Extraction
This function extracts text from uploaded PDF or TXT resumes using PyMuPDF.

It allows the model to use real resume content as context.

In [5]:
def clean_text(text):
    """
    Clean Unicode text to make it PDF-safe.
    """
    return text.encode('latin-1', 'replace').decode('latin-1')

def extract_text_from_resume(file):
    # if it's a file path (str), open directly
    if isinstance(file, str) and file.endswith(".pdf"):
        doc = fitz.open(file)
        return "\n".join(page.get_text() for page in doc)

    # if it's a Gradio upload object (temp file), use .name
    if hasattr(file, "name") and file.name.endswith(".pdf"):
        doc = fitz.open(file.name)
        return "\n".join(page.get_text() for page in doc)

    # for .txt uploads
    if hasattr(file, "name") and file.name.endswith(".txt"):
        return file.read().decode("utf-8")

    return "Unsupported file format."

### 📥 Generate Downloadable PDF Summary
Used `fpdf` to export:
- Student profile
- Career Suggestions
- Courses with **clickable links**

The entire summary can be downloaded and shared.

In [6]:
def generate_pdf(profile, careers, courses):
    pdf = FPDF()
    pdf.add_page()

    pdf.set_font("Arial", size=12)

    pdf.cell(200, 10, txt="Personalized Career Coach Summary", ln=True, align='C')
    pdf.ln(10)

    pdf.set_font("Arial", 'B', 12)
    pdf.cell(200, 10, "Student Profile", ln=True)
    pdf.set_font("Arial", size=11)
    profile_text = f"Name: {profile['name']}"
    pdf.multi_cell(0, 8, clean_text(profile_text))

    pdf.ln(10)
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(200, 10, "Career Suggestions", ln=True)
    pdf.set_font("Arial", size=11)
    for line in careers.split('\n'):
        pdf.multi_cell(0, 8, line.encode('latin-1', 'replace').decode('latin-1'))

    pdf.ln(10)
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(200, 10, "Recommended Courses", ln=True)
    pdf.set_font("Arial", size=11)
    for block in courses.strip().split('\n\n'):
        lines = block.strip().split('\n')
        if len(lines) == 2:
            career_line = lines[0]
            course_line = lines[1]

            # Extract Markdown Link
            match = re.search(r"\[(.*?)\]\((.*?)\)", course_line)
            if match:
                course_title, course_url = match.groups()
            else:
                course_title = course_line
                course_url = ""

            pdf.set_text_color(0, 0, 0)
            pdf.multi_cell(0, 8, clean_text(career_line))

            if course_url:
                pdf.set_text_color(0, 0, 255)
                pdf.set_font("Arial", 'U', 11) # underline blue
                pdf.cell(0, 10, clean_text(f"Course: {course_title}"), ln=1, link=course_url)
                pdf.set_font("Arial", '', 11)
            else:
                pdf.set_text_color(0, 0, 0)
                pdf.multi_cell(0, 8, clean_text(f"📘 Course: {course_title}"))
            pdf.ln(3)
            
    filepath = f"{profile['name'].lower().replace(' ', '_')}_careers_report.pdf"
    pdf.output(filepath)
    return filepath

### 🤖 6: GPT-3.5 Career + Course Recommendation Logic.
I used a two-step agent structure:
1. Recommend 2-3 career paths with justifications.
2. Recommend one course for each path, formatted as Markdown

This is achieved using OpenAI GPT-3.5 Turbo via chat-completion API.

In [7]:
def gpt_call(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def agent_decision_flow(profile):
    message_1 = [
        {"role": "system", "content": "You are a career counselor."},
        {"role": "user", "content": f"""
        Name: {profile['name']}
        Age: {profile['age']}
        Interests: {', '.join(profile.get('interests', []))}
        Skills: {', '.join(profile.get('skills', []))}
        Goals: {profile['goals']}

        Suggest 2-3 career paths and justify each choice briefly.
        """}
    ]
    careers = gpt_call(message_1)

    message_2 = [
        {"role": "system", "content": "You are a educational advisor."},
        {"role": "user", "content": f"""
        Based on these careers:
        
        {careers}

        Recommend one online course (from **any platform**) for each career using this format:
        Career: Career Name
        📘 Course: [Course Title](https://link)

        Only return plain text.
        """}
    ]
    courses = gpt_call(message_2)
    return careers, courses

### 🎨 7: Display Course Recommendations as HTML Cards
Instead of plain Markdown, we use HTML to create styled, clickable "cards" for each recommended course.

Each card includes:
- Career title.
- Course name.
- Clickable link to Coursera/edX/Udemy/etc

In [8]:
def format_courses_to_html(courses_text):
    blocks = courses_text.strip().split('\n\n')
    html_output = ""
    for block in blocks:
        lines = block.strip().split('\n')
        if len(lines) == 2:
            career = lines[0].replace("Career:", "").strip()
            match = re.search(r"\[(.*?)\]\((.*?)\)", lines[1])
            if match:
                title, link = match.groups()
                html_output += f"""
                <div style="border:1px solid #ccc; padding:12px; margin:10px 0; border-radius:10px; background: #f2f6ff">
                    <a href="{link}" target="_blank" style="text-decoration:none; color:#0a0a0a">
                        <b>Career:</b> {career}<br>
                        📘<b>Course:</b> {title}
                    </a>
                </div>
                """
    return html_output

### 🧠 8: Combine Everything into Main Logic
Bring together:
- Manual or resume input.
- Career + course generation.
- HTML card rendering.
- PDF export with clickable course links

This function is triggered when the user submits their profile or resume.

In [9]:
def run_career_coach(name, age, interests, skills, goals, resume_file):
    if resume_file:
        resume_text = extract_text_from_resume(resume_file)
        profile = {
            "name": name.strip() or "Student",
            "age": int(age) if age else 0,
            "interests": ["(from resume)"],
            "skills": ["(from resume)"],
            "goals": resume_text.strip()[:500]
        }
    else:
        profile = {
            "name": name.strip(),
            "age": int(age),
            "interests": [i.strip() for i in interests.split(',')],
            "skills": [s.strip() for s in skills.split(',')],
            "goals": goals.strip()
        }

    careers, courses = agent_decision_flow(profile)
    html_cards = format_courses_to_html(courses)
    pdf_path = generate_pdf(profile, careers, courses)

    return careers, html_cards, pdf_path

### 🚀 9. Launch Gradio App
The app UI includes:
- Input fields for profile or resume.
- Output for career suggestions.
- Output for HTML-based course cards.
- Downloadable PDF report.

ALL IN A FRIENDLY, 1-CLICK EXPERIENCE!

In [10]:
main_app_ui = gr.Interface(
    fn=run_career_coach,
    inputs=[
        gr.Textbox(label="Name"),
        gr.Number(label="Age", precision=0),
        gr.Textbox(label="Interests (comma-seperated)"),
        gr.Textbox(label="Skills (comma-seperated)"),
        gr.Textbox(label="Career Goals"),
        gr.File(label="Upload Resume (PDF or TXT)", file_types=[".pdf", ".txt"])
    ],
    outputs=[
        gr.Textbox(label="Career Suggestions"),
        gr.HTML(label="Recommended Courses"),
        gr.File(label="📥 Download Summary as PDF")
    ],
    title="🎓 Personalized Career Coach",
    description="Upload your resume or fill out the form to get tailored career suggestions + course recommendations. Export as a PDF!"
)

In [11]:
gr.TabbedInterface(
    [main_app_ui],
    tab_names = ["Career Coach!"]
).launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://3bc7d21c482cdaa9c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
